In [12]:
# part 1
import httpx
from selectolax.parser import HTMLParser

url = 'https://www.rei.com/c/camping-and-hiking/f/scd-deals'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

response = httpx.get(url, headers=headers)
html = HTMLParser(response.text)

def extract_text(html, sel):
    try:
        return html.css_first(sel).text()
    except AttributeError:
        return None

products = html.css('li.VcGDfKKy_dvNbxUqm29K')

n = 0
for product in products:
    n+=1
    item = {
        'id-list': n,
        'name': extract_text(product, '.Xpx0MUGhB7jSm5UvK2EY'),
        'price': extract_text(product, 'span[data-ui=sale-price]')
    }
    print(item)

{'id-list': 1, 'name': 'Flannel/Fleece Blanket', 'price': '$29.83 - $29.93'}
{'id-list': 2, 'name': 'PackLite Max 2-in-1 Power Lantern', 'price': '$34.93'}
{'id-list': 3, 'name': 'Wallace Lake Sherpa Fleece Blanket', 'price': '$44.93'}
{'id-list': 4, 'name': 'Heatbank 3 Rechargeable Hand Warmer and Power Bank', 'price': '$18.93'}
{'id-list': 5, 'name': 'The Deuce #2 UL Backcountry Trowel', 'price': '$14.93'}
{'id-list': 6, 'name': 'Pocket Torch with Refillable Lighter', 'price': '$15.93'}
{'id-list': 7, 'name': 'QuickDraw Microfilter System', 'price': '$29.93'}
{'id-list': 8, 'name': 'Luci Pro Series Inflatable Solar Lantern', 'price': '$34.73'}
{'id-list': 9, 'name': 'Folding Torch', 'price': '$47.93'}
{'id-list': 10, 'name': 'Drift Waistpack', 'price': '$39.93'}
{'id-list': 11, 'name': 'Slim-Fold Table', 'price': ' $61.83'}
{'id-list': 12, 'name': 'HeadLamp 200', 'price': '$26.73'}
{'id-list': 13, 'name': 'Thermolite Reactor Extreme Sleeping Bag Liner', 'price': '$55.93'}
{'id-list':

In [53]:
# part 2
import httpx
from selectolax.parser import HTMLParser

def get_html(baseurl, page):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    response = httpx.get(baseurl + str(page), headers=headers, follow_redirects=True)
    try:
        response.raise_for_status()
    except httpx.HTTPStatusError as exc:
        print(f'Error response {exc.response.status_code} while requesting {exc.request.url} Page Limit Exceeded.')
        return False
    html = HTMLParser(response.text)
    return html

def extract_text(html, sel):
    try:
        return html.css_first(sel).text()
    except AttributeError:
        return None

def parse_page(html):
    products = html.css('li.VcGDfKKy_dvNbxUqm29K')

    #product_list = []
    for product in products:
        item = {
            'name': extract_text(product, '.Xpx0MUGhB7jSm5UvK2EY'),
            'price': extract_text(product, 'span[data-ui=sale-price]'),
            'saving': extract_text(product, 'div[data-ui=savings-percent-variant2]')
        }
        yield item
     #   product_list.append(item)

def main():
    baseurl = 'https://www.rei.com/c/camping-and-hiking/f/scd-deals?page='
    for x in range(1,1000):
        print(f'Page {x}')
        html = get_html(baseurl, x)
        if not html:
            break
        data = parse_page(html)
        for item in data:
            print(item)

if __name__ == '__main__':
    main()

Page 1
{'name': 'Flannel/Fleece Blanket', 'price': '$29.83 - $29.93', 'saving': 'Save 50%'}
{'name': 'The Deuce #3 Heavy Duty Backcountry Trowel', 'price': '$15.93', 'saving': 'Save 27%'}
{'name': 'PackLite Max 2-in-1 Power Lantern', 'price': '$34.93', 'saving': 'Save 30%'}
{'name': 'Nalgene Sustain Graphic Wide-Mouth Water Bottle - 32 fl. oz.', 'price': ' $10.93', 'saving': None}
{'name': 'Trailgate Vehicle Shelter', 'price': '$73.83', 'saving': 'Save 50%'}
{'name': 'The Deuce #2 UL Backcountry Trowel', 'price': '$14.93', 'saving': 'Save 25%'}
{'name': 'Rambler Vacuum Pint Glass with MagSlider Lid', 'price': '$22.59', 'saving': 'Save 24%'}
{'name': 'Pocket Torch with Refillable Lighter', 'price': '$15.93', 'saving': 'Save 27%'}
{'name': 'QuickDraw Microfilter System', 'price': '$29.93', 'saving': 'Save 40%'}
{'name': 'Action Packer - 24 Gallon', 'price': '$36.93', 'saving': 'Save 26%'}
{'name': 'Drift Waistpack', 'price': '$39.93', 'saving': 'Save 27%'}
{'name': 'Trail 5 Waist Pack', 

In [79]:
# part 3
import httpx
from selectolax.parser import HTMLParser
from urllib.parse import urljoin
from dataclasses import asdict, dataclass


@dataclass
class Item:
    name: str | None
    item_num: str | None
    price: str | None
    rating: float | None


def get_html(url, **kwargs):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    if kwargs.get('page'):
        response = httpx.get(
            url + str(kwargs.get('page')), headers=headers, follow_redirects=True)
    else:
        response = httpx.get(url, headers=headers, follow_redirects=True)
    try:
        response.raise_for_status()
    except httpx.HTTPStatusError as exc:
        print(f'Error response {exc.response.status_code} while requesting {exc.request.url} Page Limit Exceeded.')
        return False
    html = HTMLParser(response.text)
    return html


def parse_search_page(html: HTMLParser):
    products = html.css('li.VcGDfKKy_dvNbxUqm29K')
    for product in products:
        yield urljoin('https://www.rei.com', product.css_first('a').attributes['href'])
    

def parse_item_page(html):
    new_item = Item(
        name=extract_text(html, 'h1#product-page-title'),
        item_num=extract_text(html, 'span#product-item-number'),
        price=extract_text(html, 'span#buy-box.product-price'),
        rating=extract_text(html, 'span.cdr-rating__number_13-5-3')
    )
    return new_item


def extract_text(html, sel):
    try:
        return html.css_first(sel).text()
    except AttributeError:
        return None


def main():
    products = []
    baseurl = 'https://www.rei.com/c/camping-and-hiking/f/scd-deals?page='
    for x in range(1,2):
        print(f'Page -> {x}')
        html = get_html(baseurl, page=x)
        if not html:
            break
        product_url = parse_search_page(html)
        for url in product_url:
            print(url)
            html = get_html(url)
            #print(html.css_first('title').text())
            products.append(parse_item_page(html))

    for product in products:
        print(asdict(product))
if __name__ == '__main__':
    main()

Page 1
https://www.rei.com/product/193589/rei-co-op-flannelfleece-blanket
https://www.rei.com/product/143797/thetentlab-the-deuce-3-heavy-duty-backcountry-trowel
https://www.rei.com/product/120697/luminaid-packlite-max-2-in-1-power-lantern
https://www.rei.com/product/203552/rei-co-op-nalgene-sustain-graphic-wide-mouth-water-bottle-32-fl-oz
https://www.rei.com/product/193121/rei-co-op-trailgate-vehicle-shelter
https://www.rei.com/product/131611/thetentlab-the-deuce-2-ul-backcountry-trowel
https://www.rei.com/product/233808/yeti-rambler-vacuum-pint-glass-with-magslider-lid
https://www.rei.com/product/148567/soto-pocket-torch-with-refillable-lighter
https://www.rei.com/product/188475/platypus-quickdraw-microfilter-system
https://www.rei.com/product/634288/rubbermaid-action-packer-24-gallon
https://www.rei.com/product/218743/mountainsmith-drift-waistpack
https://www.rei.com/product/168490/rei-co-op-trail-5-waist-pack
https://www.rei.com/product/108191/humangear-gobites-trio-utensil-set
htt

In [107]:
@dataclass
class Prueba:
    name: dict | None
    age: int

In [108]:
b = Prueba(name={'name': 'test'}, age=22)

In [109]:
for x in b.name.items():
    print(x)

('name', 'test')


In [129]:
# part 4
import httpx
from selectolax.parser import HTMLParser
from urllib.parse import urljoin
from dataclasses import asdict, dataclass, fields
import json
import csv


@dataclass
class Item:
    name: str | None
    item_num: str | None
    price: str | None
    rating: float | None


def get_html(url, **kwargs):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    if kwargs.get('page'):
        response = httpx.get(
            url + str(kwargs.get('page')), headers=headers, follow_redirects=True)
    else:
        response = httpx.get(url, headers=headers, follow_redirects=True)
    try:
        response.raise_for_status()
    except httpx.HTTPStatusError as exc:
        print(f'Error response {exc.response.status_code} while requesting {exc.request.url} Page Limit Exceeded.')
        return False
    html = HTMLParser(response.text)
    return html


def parse_search_page(html: HTMLParser):
    products = html.css('li.VcGDfKKy_dvNbxUqm29K')
    
    for product in products:
        yield urljoin('https://www.rei.com', product.css_first('a').attributes['href'])
    

def parse_item_page(html):
    new_item = Item(
        name=extract_text(html, 'h1#product-page-title'),
        item_num=extract_text(html, 'span#product-item-number'),
        price=extract_text(html, 'span#buy-box-product-price'),
        rating=extract_text(html, 'span.cdr-rating__number_13-5-3')
    )
    return asdict(new_item)


def extract_text(html, sel):
    try:
        text = html.css_first(sel).text()
        return clean_data(text)
    except AttributeError:
        return None


def export_to_json(products):
    with open('products.json', 'w', encoding='utf-8-sig') as f:
        json.dump(products, f, ensure_ascii=False, indent=4)
    print('Saved to JSON!')


def export_to_csv(products):
    field_names = [field.name for field in fields(Item)]
    
    with open('products.csv', 'w', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, field_names)
        writer.writeheader()
        writer.writerows(products)
    print('Saved to CSV!')


def append_to_csv(product):
    field_names = [field.name for field in fields(Item)]
    
    with open('appendcsv.csv', 'a', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, field_names)
        writer.writerow(product)


def clean_data(value):
    chars_to_remove = ['$', 'Item', '#']
    
    for char in chars_to_remove:
        if char in value:
            value = value.replace(char, '')
    return value.strip()


def main():
    products = []
    baseurl = 'https://www.rei.com/c/camping-and-hiking/f/scd-deals?page='
    
    for x in range(1,2):
        print(f'Page -> {x}')
        html = get_html(baseurl, page=x)
        if not html:
            break
        product_url = parse_search_page(html)
        
        for url in product_url:
            print(url)
            html = get_html(url)
            append_to_csv(parse_item_page(html))
            #print(html.css_first('title').text())
            products.append(parse_item_page(html))

    #for product in products:
    #    print(asdict(product))
    export_to_json(products)
    #export_to_csv(products)


if __name__ == '__main__':
    main()

Page -> 1
https://www.rei.com/product/193589/rei-co-op-flannelfleece-blanket
https://www.rei.com/product/143797/thetentlab-the-deuce-3-heavy-duty-backcountry-trowel
https://www.rei.com/product/120697/luminaid-packlite-max-2-in-1-power-lantern
https://www.rei.com/product/203552/rei-co-op-nalgene-sustain-graphic-wide-mouth-water-bottle-32-fl-oz
https://www.rei.com/product/193121/rei-co-op-trailgate-vehicle-shelter
https://www.rei.com/product/131611/thetentlab-the-deuce-2-ul-backcountry-trowel
https://www.rei.com/product/233808/yeti-rambler-vacuum-pint-glass-with-magslider-lid
https://www.rei.com/product/148567/soto-pocket-torch-with-refillable-lighter
https://www.rei.com/product/188475/platypus-quickdraw-microfilter-system
https://www.rei.com/product/634288/rubbermaid-action-packer-24-gallon
https://www.rei.com/product/218743/mountainsmith-drift-waistpack
https://www.rei.com/product/168490/rei-co-op-trail-5-waist-pack
https://www.rei.com/product/108191/humangear-gobites-trio-utensil-set


In [37]:
import pandas as pd

In [47]:
df = pd.DataFrame({'name': ['Jenny', 'Diana'],
                  'age': [28, 26]})

In [48]:
df

,name,age
0,Jenny,28
1,Diana,26


In [49]:
df.append({'name': ['Karla']})

AttributeError: 'DataFrame' object has no attribute 'append'

In [45]:
df

,name,age
0,Karla,28
1,Diana,26
